In [3]:
### KEY ITEMS DEFINED
#libraries
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math

#tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
#models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
#defining key models
lr = LinearRegression()
knn = KNeighborsRegressor(n_neighbors=7)
dt = DecisionTreeRegressor(max_depth=10,random_state=27)
rf = RandomForestRegressor(n_estimators=20, random_state=1)
#PCC df and lists
pcc_df = pd.DataFrame(columns=["PCC"])
mae_df = pd.DataFrame(columns=["MAE"])
rmse_df = pd.DataFrame(columns=["RMSE"])
lr_avg_pcc = []
knn_avg_pcc = []
dt_avg_pcc = []
rf_avg_pcc = []
lr_avg_mae = []
knn_avg_mae= []
dt_avg_mae = []
rf_avg_mae = []
lr_avg_rmse = []
knn_avg_rmse= []
dt_avg_rmse = []
rf_avg_rmse = []

#files
files = ['CenterHall', 'EastCampus', 'GalbraithHall', 'GeiselLibrary', 'Gilman', 'Hopkins', 'Mandeville', 'MusicBuilding', 'OttersonHall', 'PepperCanyon', 'PoliceDepartment', 'ProcessedData', 'RadyHall', 'RobinsonHall', 'SocialScience', 'StudentServices']

#workflow function
def workflow(x,y):
    #defining global variables
    global y_test
    global y_pred
    #data prep
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False)
    x_train = x_train[:130843]
    y_train = y_train[:130843]
    #create normalizer
    min_max_scaler = MinMaxScaler()
    #normalize data
    x_train_norm = min_max_scaler.fit_transform(x_train)
    x_test_norm = min_max_scaler.fit_transform(x_test)

    #LR model
    #create model
    model = lr
    #print model name
    print("Model:", model)
    #fit model to train data
    model.fit(x_train_norm, y_train)
    #predict
    y_pred = model.predict(x_test_norm)
    #reshape
    y_pred = np.array(y_pred).reshape(-1)
    y_test = np.array(y_test).reshape(-1)
    #evaluate
    corr, _ = pearsonr(y_pred, y_test)
    pcc = ('%.4f' % corr)
    lr_avg_pcc.append(pcc)
    lr_avg_mae.append(mean_absolute_error(y_test,y_pred))
    lr_avg_rmse.append(math.sqrt(mean_squared_error(y_test,y_pred)))

    #KNN model
    #create model
    model = knn
    #print model name
    print("Model:", model)
    #fit model to train data
    model.fit(x_train_norm, y_train)
    #predict
    y_pred = model.predict(x_test_norm)
    #reshape
    y_pred = np.array(y_pred).reshape(-1)
    y_test = np.array(y_test).reshape(-1)
    #evaluate
    corr, _ = pearsonr(y_pred, y_test)
    pcc = ('%.4f' % corr)
    knn_avg_pcc.append(pcc)
    knn_avg_mae.append(mean_absolute_error(y_test,y_pred))
    knn_avg_rmse.append(math.sqrt(mean_squared_error(y_test,y_pred)))

    #DT model
    #create model
    model = dt
    #print model name
    print("Model:", model)
    #fit model to train data
    model.fit(x_train_norm, y_train)
    #predict
    y_pred = model.predict(x_test_norm)
    #reshape
    y_pred = np.array(y_pred).reshape(-1)
    y_test = np.array(y_test).reshape(-1)
    #evaluate
    corr, _ = pearsonr(y_pred, y_test)
    pcc = ('%.4f' % corr)
    dt_avg_pcc.append(pcc)
    dt_avg_mae.append(mean_absolute_error(y_test,y_pred))
    dt_avg_rmse.append(math.sqrt(mean_squared_error(y_test,y_pred)))

    #RF model
    #create model
    model = rf
    #print model name
    print("Model:", model)
    #fit model to train data
    model.fit(x_train_norm, y_train)
    #predict
    y_pred = model.predict(x_test_norm)
    #reshape
    y_pred = np.array(y_pred).reshape(-1)
    y_test = np.array(y_test).reshape(-1)
    #evaluate
    corr, _ = pearsonr(y_pred, y_test)
    pcc = ('%.4f' % corr)
    rf_avg_pcc.append(pcc)
    rf_avg_mae.append(mean_absolute_error(y_test,y_pred))
    rf_avg_rmse.append(math.sqrt(mean_squared_error(y_test,y_pred)))

In [4]:
for file in files:
    #loading data
    df = pd.read_csv('/Users/farihahahmed/Desktop/BuildingLoad/%s.csv' % file) 
    #delete unnecessary datetime column
    df.drop('DateTime', inplace=True, axis=1)
    #rename RealPower column to Present-1
    df.rename(columns = {'RealPower':'Present-1'}, inplace = True)
    #create duplicate columns
    df['Present-2'] = df['Present-1']
    df['Present-3'] = df['Present-1']
    df['Present-4'] = df['Present-1']
    df['Future-1'] = df['Present-1']
    df['Future-2'] = df['Present-1']
    df['Future-3'] = df['Present-1']
    df['Future-4'] = df['Present-1']
    #shift up rows
    df['Present-2'] = df['Present-2'].shift(-1)
    df['Present-3'] = df['Present-3'].shift(-2)
    df['Present-4'] = df['Present-4'].shift(-3)
    df['Future-1'] = df['Future-1'].shift(-4)
    df['Future-2'] = df['Future-2'].shift(-5)
    df['Future-3'] = df['Future-3'].shift(-6)
    df['Future-4'] = df['Future-4'].shift(-7)
    #delete last 7 rows
    df = df.head(df.shape[0] -7)
    #interpolate if there are any missing values
    df = df.interpolate(method ='linear', limit_direction ='forward')
    #create input and output
    x = df.loc[:, ['Present-1', 'Present-2', 'Present-3', 'Present-4']]
    y = df.loc[:, ['Future-1', 'Future-2', 'Future-3', 'Future-4']]
    #workflow
    workflow(x,y)

Model: LinearRegression()
Model: KNeighborsRegressor(n_neighbors=7)
Model: DecisionTreeRegressor(max_depth=10, random_state=27)
Model: RandomForestRegressor(n_estimators=20, random_state=1)
Model: LinearRegression()
Model: KNeighborsRegressor(n_neighbors=7)
Model: DecisionTreeRegressor(max_depth=10, random_state=27)
Model: RandomForestRegressor(n_estimators=20, random_state=1)
Model: LinearRegression()
Model: KNeighborsRegressor(n_neighbors=7)
Model: DecisionTreeRegressor(max_depth=10, random_state=27)
Model: RandomForestRegressor(n_estimators=20, random_state=1)
Model: LinearRegression()
Model: KNeighborsRegressor(n_neighbors=7)
Model: DecisionTreeRegressor(max_depth=10, random_state=27)
Model: RandomForestRegressor(n_estimators=20, random_state=1)
Model: LinearRegression()
Model: KNeighborsRegressor(n_neighbors=7)
Model: DecisionTreeRegressor(max_depth=10, random_state=27)
Model: RandomForestRegressor(n_estimators=20, random_state=1)
Model: LinearRegression()
Model: KNeighborsRegress

In [5]:
pcc_df.loc['LR']=np.mean(np.array(lr_avg_pcc).astype(float))
pcc_df.loc['KNN']=np.mean(np.array(knn_avg_pcc).astype(float))
pcc_df.loc['DT']=np.mean(np.array(dt_avg_pcc).astype(float))
pcc_df.loc['RF']=np.mean(np.array(rf_avg_pcc).astype(float))
print(pcc_df)

          PCC
LR   0.936462
KNN  0.902744
DT   0.838675
RF   0.897512


In [6]:
mae_df.loc['LR']=np.mean(np.array(lr_avg_mae).astype(float))
mae_df.loc['KNN']=np.mean(np.array(knn_avg_mae).astype(float))
mae_df.loc['DT']=np.mean(np.array(dt_avg_mae).astype(float))
mae_df.loc['RF']=np.mean(np.array(rf_avg_mae).astype(float))
print(mae_df)

           MAE
LR   17.326565
KNN  14.989773
DT   15.786722
RF   14.813555


In [7]:
rmse_df.loc['LR']=np.mean(np.array(lr_avg_rmse).astype(float))
rmse_df.loc['KNN']=np.mean(np.array(knn_avg_rmse).astype(float))
rmse_df.loc['DT']=np.mean(np.array(dt_avg_rmse).astype(float))
rmse_df.loc['RF']=np.mean(np.array(rf_avg_rmse).astype(float))
print(rmse_df)

          RMSE
LR   20.722373
KNN  18.061679
DT   19.486463
RF   17.877243
